In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QLabel, QFileDialog, QInputDialog, QSizePolicy, QSlider
from PyQt5.QtGui import QPixmap, QIcon, QImage, QPainter, QPen, QFont
from PyQt5.QtCore import QSize, Qt, QPoint, QRect
import cv2
import sys
import numpy as np

class MyApp(QWidget): # 변수 선언 
    def __init__(self):
        super().__init__()
        self.img = None
        self.original_img = None
        self.drawing_mode = False
        self.text = None
        self.drawing = False
        self.original_img=None
        self.cropping = False
        self.crop_start = QPoint()
        self.crop_end = QPoint()
        self.mosaic_mode = False
        self.mosaic_start = QPoint()
        self.mosaic_end = QPoint()
        self.brush_color = [(0, 0, 0), (255, 0, 0), (0, 255, 0), (0, 0, 255)]  # black, red, green, blue
        self.brush_color_index = 0
        self.initSliders()
        self.initUI()

    def initUI(self): #초기 ui 설정
        self.layout = QVBoxLayout()
        self.setLayout(self.layout)

        self.label = QLabel()
        self.label.setAlignment(Qt.AlignCenter)  
        self.layout.addWidget(self.label)

        self.hbox_layout1 = QHBoxLayout()
        self.hbox_layout2 = QHBoxLayout() 
        #버튼에 이미지 삽입
        self.buttons = ['import', 'save', 'filter','rotate', 'flip', 'draw','reset', 'text', 'crop', 'canny', 'mosaic','star','heart','collage']
        for idx, button in enumerate(self.buttons):
            btn = QPushButton(button)
            btn.setIcon(QIcon(f'icon/{button}.png'))
            btn.setIconSize(QSize(50, 50)) #버튼 크기
            btn.clicked.connect(self.click_buttons) #버튼 연결
            if idx < 7: 
                self.hbox_layout1.addWidget(btn)
            else:
                self.hbox_layout2.addWidget(btn)  
        self.layout.addLayout(self.hbox_layout1)# 2열로 버튼 
        self.layout.addLayout(self.hbox_layout2)
             
        #필터함수에서 사용하는 트랙바 위젯 추가
        self.layout.addWidget(self.brightness_slider)
        self.layout.addWidget(self.blur_slider)
        self.layout.addWidget(self.sharpness_slider)

        #슬라이더를 처음에 숨김
        self.brightness_slider.hide()
        self.blur_slider.hide()
        self.sharpness_slider.hide()

        self.setWindowTitle('Photo Album')
        self.setGeometry(300, 300, 300, 200)
        self.show()
        
    def initSliders(self): # 트랙바(slider)구현 
        self.brightness_slider = QSlider(Qt.Horizontal) #선언
        self.brightness_slider.setRange(0, 100) #0에서 100사이의 값
        self.brightness_slider.setValue(0) #초기값
        self.brightness_slider.valueChanged.connect(self.apply_filters) #필터 적용

        self.blur_slider = QSlider(Qt.Horizontal)
        self.blur_slider.setRange(0, 100)
        self.blur_slider.setValue(0)
        self.blur_slider.valueChanged.connect(self.apply_filters)

        self.sharpness_slider = QSlider(Qt.Horizontal)
        self.sharpness_slider.setRange(1, 100)
        self.sharpness_slider.setValue(1)
        self.sharpness_slider.valueChanged.connect(self.apply_filters)

        

    def click_buttons(self): #버튼을 누르면 실행할 내용을 적음 
        button_text = self.sender().text()
        if button_text == 'import':
            fname = QFileDialog.getOpenFileName(self)
            if fname[0]:
                self.open_image(fname)
        elif button_text == 'save':
            fname = QFileDialog.getSaveFileName(self, 'Save File', '', "Images (*.png *.xpm *.jpg *.bmp)")
            if fname[0]:
                cv2.imwrite(fname[0], self.img)
        elif button_text == 'rotate':
            self.rotate_image()
        elif button_text == 'flip':
            self.flip_image()
        elif button_text == 'reset':
            self.reset_image()
        elif button_text == 'text':
            self.drawing_mode = False
            self.cropping = False
            self.mosaic_mode = False
            self.text, ok = QInputDialog.getText(self, 'Text Input', 'Enter your text:')
        elif button_text == 'canny':
            self.canny_image()
        elif button_text=='filter':
            self.toggle_sliders()
        elif button_text=='draw':
            self.drawing_mode = True 
            self.cropping = False
            self.mosaic_mode = False
            self.stamp_mode = False
        elif button_text=='crop':
            self.drawing_mode = False  
            self.cropping = True 
            self.mosaic_mode = False
            self.stamp_mode = False
        elif button_text == 'mosaic':
            self.drawing_mode = False  
            self.cropping = False 
            self.mosaic_mode = True
            self.stamp_mode = False
        elif button_text == 'heart':
            self.drawing_mode = False
            self.cropping = False
            self.mosaic_mode = False
            self.stamp_mode = True
            self.stamp = cv2.imread('icon/heart.png', cv2.IMREAD_UNCHANGED)
            self.stamp = cv2.resize(self.stamp, (50, 50))
        elif button_text == 'star':
            self.drawing_mode = False
            self.cropping = False
            self.mosaic_mode = False
            self.stamp_mode = True
            self.stamp = cv2.imread('icon/star.png', cv2.IMREAD_UNCHANGED)
            self.stamp = cv2.resize(self.stamp, (50, 50))
        elif button_text == 'collage':
            self.collage_images()
        #drawing과 crop과 mosaic와 stamp는 같은 마우스 이벤트 함수를 사용하므로 false, true로 구분
            
            
    def open_image(self, fname): #이미지 불러오기 함수
        self.img = cv2.imread(fname[0])
        self.original_img = self.img.copy() #이미지를 초기화할때 사용하기 위해 copy
        self.display_image(self.img) #불러온 이미지가 화면에 보여지도록 함

    def display_image(self,img): #이미지를 화면에 나타내는 함수
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #cv2.imread는 bgr형태로 로딩하기 때문에 bgr에서 rgb 형태로 변환시킴
        qimg = QImage(img_rgb.data, img_rgb.shape[1], img_rgb.shape[0], img_rgb.strides[0], QImage.Format_RGB888)
        self.label.setPixmap(QPixmap.fromImage(qimg))

    def rotate_image(self): #이미지를 회전시키는 함수
        self.img = cv2.rotate(self.img, cv2.ROTATE_90_CLOCKWISE) #cv2의 rotate 함수를 이용해 버튼을 클릭할때마다 90도씩 회전
        self.display_image(self.img) 

    def flip_image(self): #좌우반전하는 함수
        self.img = cv2.flip(self.img, 1) #cv2의 flip 함수를 사용해 이미지를 좌우반전 시킴
        self.display_image(self.img)

    def reset_image(self): #이미지를 초기화하는 함수
        self.img = self.original_img.copy() #원본이미지를 토대로 이미지를 초기화
        self.display_image(self.img)

    def mousePressEvent(self, event):
        label_size = self.label.size()
        pixmap_size = self.label.pixmap().size()

        x = event.pos().x() - (label_size.width() - pixmap_size.width()) // 2
        y = event.pos().y() - (label_size.height() - pixmap_size.height()) // 2

        if 0 <= x < pixmap_size.width() and 0 <= y < pixmap_size.height():
            if event.button() == Qt.LeftButton:
                if self.drawing_mode:  # 그림 그리기 모드
                    self.drawing=True
                    self.last_point = QPoint(x, y)
                elif self.cropping: #자르기 모드
                    self.crop_start = QPoint(x,y)
                elif self.mosaic_mode:  # 모자이크 모드
                    self.mosaic_start = QPoint(x, y)
                elif self.stamp_mode:  # 스탬프 모드
                    h, w = self.stamp.shape[:2]
                    if 0 <= x - w//2 < self.img.shape[1] and 0 <= y - h//2 < self.img.shape[0] and 0 < x + w//2 <= self.img.shape[1] and 0 < y + h//2 <= self.img.shape[0]:
                        self.img = self.apply_stamp(self.img, self.stamp, (x - w//2, y - h//2)) 
                        self.display_image(self.img)
                else:  # 텍스트 그리기 모드
                    if self.text is not None:
                        self.img = cv2.putText(self.img, self.text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2) #putText 함수 활용
                        self.display_image(self.img)
            elif event.button() == Qt.RightButton: #우클릭할때 펜 색상 변경 
                self.brush_color_index = (self.brush_color_index + 1) % len(self.brush_color)

    def mouseMoveEvent(self, event):
        label_size = self.label.size()
        pixmap_size = self.label.pixmap().size()

        x = event.pos().x() - (label_size.width() - pixmap_size.width()) // 2
        y = event.pos().y() - (label_size.height() - pixmap_size.height()) // 2

        if 0 <= x < pixmap_size.width() and 0 <= y < pixmap_size.height():
            if self.drawing: #그리기 모드
                self.img = cv2.line(self.img, (self.last_point.x(), self.last_point.y()), (x, y), self.brush_color[self.brush_color_index], 2)
                self.last_point = QPoint(x, y)
                self.display_image(self.img) #cv2.line 함수를 통해 그리기를 진행하고 화면에 보이도록 함
            if self.cropping: #자르기 모드
                self.crop_end = QPoint(x, y)
                self.temp_img = self.img.copy() # cv2.rectangle 함수를 통해 설정하는 roi를 화면에 보이도록 함
                cv2.rectangle(self.temp_img, (self.crop_start.x(), self.crop_start.y()), (x, y), (0, 255, 0), 2)
                self.display_image(self.temp_img)
            if self.mosaic_mode: #모자이크 모드
                self.mosaic_end = QPoint(x, y)
                self.temp_img = self.img.copy() #cv2. rectangle 함수를 통해 설정하는 roi를 화면에 보이도록 함
                cv2.rectangle(self.temp_img, (self.mosaic_start.x(), self.mosaic_start.y()), (x, y), (0, 255, 0), 2)
                self.display_image(self.temp_img)
    def mouseReleaseEvent(self, event):
        label_size = self.label.size()
        pixmap_size = self.label.pixmap().size()

        x = event.pos().x() - (label_size.width() - pixmap_size.width()) // 2
        y = event.pos().y() - (label_size.height() - pixmap_size.height()) // 2

        if self.drawing and (event.button() == Qt.LeftButton):
            self.drawing = False
        if self.cropping:
            x_start, y_start = min(self.crop_start.x(), self.crop_end.x()), min(self.crop_start.y(), self.crop_end.y()) #start와 end 설정
            x_end, y_end = max(self.crop_start.x(), self.crop_end.x()), max(self.crop_start.y(), self.crop_end.y())
            if 0 <= x_start < self.img.shape[1] and 0 <= y_start < self.img.shape[0] and 0 < x_end <= self.img.shape[1] and 0 < y_end <= self.img.shape[0]:
                self.img = self.img[y_start:y_end, x_start:x_end] #roi 범위만큼 자르기
                if self.img.size > 0:
                    self.display_image(self.img) #이미지 출력
            self.cropping = False #자르기 끝나면 모드 변경
        if self.mosaic_mode:
            x_start, y_start = min(self.mosaic_start.x(), self.mosaic_end.x()), min(self.mosaic_start.y(), self.mosaic_end.y())
            x_end, y_end = max(self.mosaic_start.x(), self.mosaic_end.x()), max(self.mosaic_start.y(), self.mosaic_end.y())
            w, h = x_end - x_start, y_end - y_start # width, height 설정
            if 0 <= x_start < self.img.shape[1] and 0 <= y_start < self.img.shape[0] and 0 < x_end <= self.img.shape[1] and 0 < y_end <= self.img.shape[0]:
                roi = self.img[y_start:y_end, x_start:x_end]
                roi = cv2.resize(roi, (w//15, h//15)) #1/rate비율로 축소
                roi = cv2.resize(roi, (w,h), interpolation=cv2.INTER_AREA)  #원래 크기로 확대
                self.img[y_start:y_end, x_start:x_end] = roi
                self.display_image(self.img)
            self.mosaic_mode = False
    def apply_stamp(self, background, stamp, position=(0, 0)):
        stamp_h, stamp_w, stamp_c = stamp.shape # 스탬프 이미지의 정보를 가져오기
        roi = background[position[1]:position[1]+stamp_h, position[0]:position[0]+stamp_w] # ROI 설정 (스탬프 이미지의 크기와 동일하게 설정).
        mask = stamp[:, :, 3] 
        mask_inv = cv2.bitwise_not(mask) # 마스크 생성
        bg_bg = cv2.bitwise_and(roi, roi, mask=mask_inv) #roi에 스탬프 이미지 합성

        if 0 <= position[0] < background.shape[1] and 0 <= position[1] < background.shape[0]:
            fg_fg = cv2.resize(stamp[:, :, :3], (bg_bg.shape[1], bg_bg.shape[0])) #스탬프 이미지 크기조절

            # 합성한 이미지를 원본 이미지에 반영
            dst = cv2.addWeighted(bg_bg, 1, fg_fg, 1, 0)
            background[position[1]:position[1]+stamp_h, position[0]:position[0]+stamp_w] = dst

        return background
    
    def canny_image(self): # 캐니 검출을 통해 이미지의 외곽선을 만드는 함수
        blur_img = cv2.GaussianBlur(self.img, (5, 5), cv2.BORDER_DEFAULT) #가우시안 블러링을 통한 노이즈 제거
        self.img = 255 - cv2.Canny(blur_img, 30, 60) #opencv 캐니 에지 검출
        self.img = cv2.cvtColor(self.img, cv2.COLOR_GRAY2RGB)
        self.display_image(self.img)
        
    def toggle_sliders(self): #필터에 사용되는 트랙바가 나오는 함수
        if self.brightness_slider.isVisible(): #현재 보여지고 있다면 숨기고
            self.brightness_slider.hide()
            self.blur_slider.hide()
            self.sharpness_slider.hide()
        else: #아니라면 화면에 보여지도록
            self.brightness_slider.show()
            self.blur_slider.show()
            self.sharpness_slider.show()
    def apply_filters(self): #필터를 적용하는 함수
        self.img = self.original_img.copy()
        # 밝기 조절 (opencv의 saturation 방식을 활용 행렬에 값을 더하여 밝아지도록 함)
        self.img = cv2.add(self.img, np.ones(self.img.shape, dtype=np.uint8) * self.brightness_slider.value())

        # 블러 조절
        if self.blur_slider.value() > 0: #cv2 blur 함수를 사용하여 진행
            self.img = cv2.blur(self.img, (self.blur_slider.value(), self.blur_slider.value()))

        # 선명도 조절
        if self.sharpness_slider.value() != 1:
            blurred = cv2.GaussianBlur(self.img, (0, 0), self.sharpness_slider.value())
            self.img = cv2.addWeighted(self.img, 1.5, blurred, -0.5, 0, self.img)

        self.display_image(self.img)
    def collage_images(self):
        
        collage_imgs = []  

        for _ in range(4):
            fname = QFileDialog.getOpenFileName(self)
            if fname[0]:
                img = cv2.imread(fname[0])
                collage_imgs.append(img) #4번동안 이미지를 선택하여 배열에 저장

        if len(collage_imgs) == 4:
            collage_imgs_resized = [cv2.resize(img, (200, 200)) for img in collage_imgs] #이미지의 크기를 조절
            collage = np.zeros((400, 400, 3), dtype=np.uint8) #빈 이미지 생성
            collage[:200, :200] = collage_imgs_resized[0] #위치에 맞게 배치
            collage[:200, 200:] = collage_imgs_resized[1]
            collage[200:, :200] = collage_imgs_resized[2]
            collage[200:, 200:] = collage_imgs_resized[3]

            self.img = collage
            self.display_image(self.img)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MyApp()
    sys.exit(app.exec_())

In [2]:
stamp = cv2.imread(r'C:\Users\Administrator\Desktop\icon\heart.png', cv2.IMREAD_UNCHANGED)
cv2.imshow('stamp', stamp)
cv2.waitKey(0)
cv2.destroyAllWindows()
